# POOR6

Group project of the "Database 2" course, Computer Engineering MD, Università degli Studi di Padova, a.a. 2023/24, by the POOR6 group: Merlo Simone, Gobbo Riccardo, Spinosa Diego.

#### Introduction

We have chosen to work on three tightly linked **datasets**:
 - "*Electric Vehicle Population Data*" provided by the Dept. of Licensing of the State of Washington, US. This dataset shows the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through Washington State Department of Licensing (DOL).  [source](https://data.wa.gov/Transportation/Electric-Vehicle-Population-Data/f6w7-q2d2)
 - "*Alternative Fuel Stations*", as part of the Alternative Fuels Data Center of the U.S. Department of Energy. This dataset was used to obtain location and properties of all the registered EV charging stations in the state of Washington. [source](https://afdc.energy.gov/data_download/alt_fuel_stations_format)
 - "*SOI Tax Stats - Individual Income Tax Statistics*" provided by the IRS (Internal Revenue Service) of the US. Data are based on individual income tax returns filed with the IRS and are used to determine how the average income varies among Washington's zipcodes. [source](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2020-zip-code-data-soi)

The project aims at linking these 3 data sources to discover whether there are some interesting correlations between EV diffusion, EV charging station density and average income in the scope of the territory of state of Washington. We are focusing on this region since we have found a particularly good data quality and availability, and also because its scenario can resemble the one found in most other US states or even other first world countries worldwide.

In order to properly link these datasets we exploited a fourth data source that provided us data regarding the ZipCodes of the state of Washington.

## NOTES:
In our domain we assumed that each ZipCode is related to one and one only city, in the reality a ZipCodes can span over multiple cities but each of them has a "primary city". We considered the "primary city" as the only city related to a particular ZipCode.

# Data processing

### Importing packages

In [67]:
import os
import shutil as sh
from pathlib import Path
from unidecode import unidecode

### Functions definition
Defining a function to turn text strings in something usable for IRIs.
In particular we first map the string's characters to ASCII only characters, then we convert the string in Camel-case notation and eventually we remove spaces.

In [68]:
def urify_string(s: str):
    s=unidecode(s)
    pattern = "[^0-9a-zA-Z\s]+"
    s = re.sub(pattern, " ", s).title().replace(" ", "")
    return s

### Preliminary operations

Creating paths and folder structure

In [69]:
DATA_FOLDER = "clean_data"
SOURCE_FOLDER = "src_data/"
OUTPUT_FOLDER = "output"
STATIONS_FILE = "stations_pub+priv_open.csv"
LOCATIONS = SOURCE_FOLDER + "wa_zips_cities_counties.csv"
CARS_FILE = SOURCE_FOLDER + "Electric_Vehicle_Population_Data.csv"
WAGE_FILE = SOURCE_FOLDER + "20zpallnoagi.csv"

# Get the absolute path
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
try:
    # Remove the existing folder
    sh.rmtree(DATA_FOLDER)
    sh.rmtree(OUTPUT_FOLDER)
except FileNotFoundError:
    print("--- No folder to remove ---")

# Create new folder for clean data
os.mkdir(DATA_FOLDER)
print(f"Folder '{DATA_FOLDER}' created.")
os.mkdir(OUTPUT_FOLDER)
print(f"Folder '{OUTPUT_FOLDER}' created.")

DATA_FOLDER += "/"
OUTPUT_FOLDER += "/"

Folder 'clean_data' created.
Folder 'output' created.


## Data cleaning

#### Correcting and filtering fuel stations file

In [70]:
file = open(SOURCE_FOLDER + STATIONS_FILE, "r", encoding="utf-8")                  # Input file
wa_fuel_stations = wa_e_stations = open(DATA_FOLDER + STATIONS_FILE, "w", encoding="utf-8")     # Output file

# Write CSV headers
wa_fuel_stations.write(file.readline())

row = file.readline()               # Read first line
while (row2 := file.readline()):
    row2_error = False              

    # If the row is interrupted, recover it (there can be multiple interruption)
    while("ELEC" not in row2):
        row2_error = True
        index = row2.find('",')                                     # Find the end of last interrupted string, if exists
        row = row.strip() + row2[index if index != -1 else 0 : ]    # Concatenate the row begin with the second part
        row2 = file.readline()

    if ",WA," in row: wa_e_stations.write(row)
    row = row2                                                      # Check on next cycle

file.close()
wa_e_stations.close()

# RDF creation

### Importing packages

In [71]:
import pandas as pd
from urllib.parse import quote
from rdflib import Graph, Literal, RDF, RDFS, URIRef, Namespace
from rdflib.namespace import XSD
import re

## ZipCode codes, cities, counties
Reading the ZipCode file and creating the triples related to ZipCodes,Cities and Counties

In [72]:
places = pd.read_csv(LOCATIONS, sep=",")

ECO = Namespace("http://www.dei.unipd.it/~poor6/db2/ontologies/2023/electricCars#")

graph = Graph()
graph.bind("elec", ECO)

In [73]:
%%time

for index, row in places.iterrows():
    #Creating the URIs using the urify_string function
    ZipCode = URIRef(ECO[str(row['Zipcode'])])
    City = URIRef(ECO[urify_string(str(row['City']))])
    County = URIRef(ECO[urify_string(str(row['County']))])

    graph.add((ZipCode, RDF.type, ECO.ZipCode)) #Adding ZipCodes
    graph.add((City, RDF.type, ECO.City)) #Adding City
    graph.add((County, RDF.type, ECO.County)) #Adding County
    
    #Adding labels to keep the original strings
    graph.add((City, RDFS.label, Literal(str(row['City']), datatype=XSD.string)))
    graph.add((County, RDFS.label, Literal(str(row['County']), datatype=XSD.string)))

    #Adding relations
    graph.add((ZipCode, ECO["ofCity"], City))
    graph.add((City, ECO["belongsTo"], County))

CPU times: total: 93.8 ms
Wall time: 103 ms


In [74]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'locations.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 93.8 ms
Wall time: 98.7 ms


## Washington electrical stations

Reading the Electric StationsZip file and creating the triples related to them.

In [84]:
stations = pd.read_csv(DATA_FOLDER + STATIONS_FILE, sep=",")

stations.info()

graph = Graph()
graph.bind("elec", ECO)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2188 entries, 0 to 2187
Data columns (total 74 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Fuel Type Code                           2188 non-null   object 
 1   Station Name                             2188 non-null   object 
 2   Street Address                           2185 non-null   object 
 3   Intersection Directions                  109 non-null    object 
 4   City                                     2188 non-null   object 
 5   State                                    2188 non-null   object 
 6   ZIP                                      2188 non-null   object 
 7   Plus4                                    0 non-null      float64
 8   Station Phone                            2028 non-null   object 
 9   Status Code                              2188 non-null   object 
 10  Expected Date                            0 non-n

In [85]:
%%time

for index, row in stations.iterrows():
    Station = URIRef(ECO[str(index)])     # Create node (prefix + incremental id)

    #Adding the statio type (private or public)
    if(re.search(".*[Pp]rivate.*",row['Access Code'])):
        graph.add((Station, RDF.type, ECO.PrivateStation))
    elif (re.search(".*[Pp]ublic.*",row['Access Code'])):
        graph.add((Station, RDF.type, ECO.PublicStation))
    
    graph.add((Station, RDF.type, ECO.Station))             

    # Adding station name and address if present
    graph.add((Station, ECO['hasName'], Literal(row['Station Name'], datatype=XSD.string)))
    if row['Street Address']:
        graph.add((Station, ECO['hasAddress'], Literal(row['Street Address'], datatype=XSD.string)))
    
    latitude = None
    longitude = None
    latitude = row['Latitude']
    longitude = row['Longitude']

    if latitude is not None and longitude is not None:
        graph.add((Station, ECO['hasLatitude'], Literal(row['Latitude'], datatype=XSD.float)))
        graph.add((Station, ECO['hasLongitude'], Literal(row['Longitude'], datatype=XSD.float)))

    #Avoiding strange cases
    if " " in row['ZIP']: 
        print(f"Error in ZIP '{row['ZIP']}', skipped")
        continue

    #Linking a station to its ZipCode
    ZipCode = URIRef(ECO[row['ZIP']])
    graph.add((Station, ECO['locatedIn'], ZipCode))


Error in ZIP 'G9N 0', skipped
CPU times: total: 453 ms
Wall time: 453 ms


In [86]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'stations.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 375 ms
Wall time: 375 ms


## Cars
Reading the cars file and creating the triples related to them. This includes the creation of cars, models, makers and model-years

In [78]:
cars = pd.read_csv(CARS_FILE, sep=",")
cars.info()

graph = Graph()      #Graph redefine, so to keep separate working spaces.
graph.bind("elec", ECO)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150482 entries, 0 to 150481
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         150482 non-null  object 
 1   County                                             150479 non-null  object 
 2   City                                               150479 non-null  object 
 3   State                                              150482 non-null  object 
 4   Postal Code                                        150479 non-null  float64
 5   Model Year                                         150482 non-null  int64  
 6   Make                                               150482 non-null  object 
 7   Model                                              150482 non-null  object 
 8   Electric Vehicle Type                              150482 non-null  object

In [79]:
%%time


for index, row in cars.iterrows():
    
    #Skipping cars that are outside washington state (if present)
    if('WA' != row['State']):
        continue
        
    Car = URIRef(ECO[str(row['DOL Vehicle ID'])])     # Create car based on the DOL 
    
    #Creting a string ready to be turned into a uri
    SpacelessModel = urify_string(row['Model'])
    
    #Creatign modelYear
    ModelYear = URIRef(ECO[ SpacelessModel + str(row['Model Year']) ])  #2012 'Grand Cherokee' => GrandCherokee2012
    
    #Linking a car to its ZipCode
    ZipCode = URIRef(ECO[str(int(row['Postal Code']))])
    graph.add((Car, ECO['isRegisteredIn'], ZipCode))
    
    # If there are valid coordinates
    if (row['Vehicle Location'] is not None) and (point := re.findall("(?<=\().*(?=\))", str(row['Vehicle Location']))):
        coordinates = point.pop().split()
        latitude = coordinates.pop()
        longitude = coordinates.pop()
        # Cars' coordinates represent the ZipCode centers, hence when these information is retrieved, it's added to the corresponding ZipCode
        graph.add((ZipCode, ECO['hasLongitude'], Literal(longitude, datatype=XSD.float)))
        graph.add((ZipCode, ECO['hasLatitude'], Literal(latitude, datatype=XSD.float)))

    # Car-instance specific insertions
    
    graph.add((Car, RDF.type, ECO.Car))
    graph.add((Car, ECO['hasVIN'], Literal(row['VIN (1-10)'], datatype=XSD.string) ))
        
    
    #Adding car type (BEV/PHEV or CleanCar/NotCleanCar)
    if(re.search(".*BEV.*",row['Electric Vehicle Type'])):
        graph.add((Car, RDF.type, ECO.CleanCar))
        graph.add((Car, RDF.type, ECO.BEV))
    else:
        graph.add((Car, RDF.type, ECO.PHEV))
        if ((row['Electric Range'] >0) and (row['Electric Range'] <30)):
            graph.add((Car, RDF.type, ECO.NotCleanCar))
        elif ((row['Electric Range'] >=30)):
            graph.add((Car, RDF.type, ECO.CleanCar))
        

    
    
    #Adding range only if maintained (grater than zero)
    if (row['Electric Range'] > 0):
        graph.add((Car, ECO['hasRange'], Literal(row['Electric Range'], datatype=XSD.integer) ))
    
    graph.add((Car, ECO['hasModelYear'], ModelYear))
    
    graph.add((ModelYear, RDF.type, ECO.ModelYear))
    graph.add((ModelYear, ECO['hasYear'], Literal(row['Model Year'], datatype=XSD.gYear) ))
    
    #Adding MSRP only if maintained
    if (row['Base MSRP'] > 0):
        graph.add((ModelYear, ECO['hasMSRP'], Literal(row['Base MSRP'], datatype=XSD.integer) ))
    
    Model = URIRef(ECO[SpacelessModel])  #<--- uri-ready string defined before
    graph.add((Model, RDFS.label, Literal(row['Model'], datatype=XSD.string))) #LABEL: TO SAVE ORIG. SPACED MODEL
    graph.add((ModelYear, ECO['ofModel'], Model))
    graph.add((Model, RDF.type, ECO.Model))
 
    Maker = URIRef(ECO[urify_string(row['Make'])])   #<--- spaceless uri also for maker (ex. "Aston Martin")
    graph.add((Maker, RDFS.label, Literal(row['Make'], datatype=XSD.string))) #LABEL: TO SAVE ORIG. SPACED MAKER
    graph.add((Model, ECO['madeBy'], Maker))
    graph.add((Maker, RDF.type, ECO.Maker))
        
    
    

CPU times: total: 49.8 s
Wall time: 50.3 s


In [80]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'cars.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))


--- saving serialization ---
CPU times: total: 24.9 s
Wall time: 25.1 s


## Wage per ZIP code
Reading the AGI file and creating the triples related to AGI.

In [81]:
salaries = pd.read_csv(WAGE_FILE, sep=",")
salaries.info()

graph = Graph()      #Graph redefine, so to keep separate working spaces.
graph.bind("elec", ECO)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27744 entries, 0 to 27743
Columns: 165 entries, STATEFIPS to A12000
dtypes: float64(161), int64(3), object(1)
memory usage: 34.9+ MB


In [82]:
%%time

for index, row in salaries.iterrows():
    # Exclude non Washington data
    if "WA" not in row['STATE'] or '00000' in str(row['ZIPCODE']) or '99999' in str(row['ZIPCODE']) or '0' == str(row['ZIPCODE']):
        continue

    Zipcode = URIRef(ECO[str(row['ZIPCODE'])])
    
    agi = int(((row['A00100'])/float(row['N2'])) * 1000)

    graph.add((Zipcode, ECO['hasAgi'], Literal(agi, datatype=XSD.int)))

CPU times: total: 766 ms
Wall time: 797 ms


In [83]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(OUTPUT_FOLDER + 'agi.ttl', 'w') as file:
    file.write(graph.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 46.9 ms
Wall time: 25.9 ms
